In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df1= pd.read_csv("Desktop/emoint tweet/fear-ratings-0to1.train.txt", delimiter='\t', header=None)
df1.columns = ['Id', 'tweet', 'emotion', 'score']

In [3]:
print(df1)


         Id                                              tweet emotion  score
0     20000  I feel like I am drowning. #depression #anxiet...    fear  0.979
1     20001  I get so nervous even thinking about talking t...    fear  0.979
2     20002                     I lost my blinders .... #panic    fear  0.975
3     20003  I feel like I am drowning. #depression  #falur...    fear  0.938
4     20004  This is the scariest American Horror Story out...    fear  0.938
...     ...                                                ...     ...    ...
1142  21142     Pull over #tonight and make your car #shake 😋💦    fear  0.104
1143  21143  @Melanie_Pierce @HunterHayes awe ain't he a sw...    fear  0.083
1144  21144         @FraserKeegan just had a steak pie supper     fear  0.083
1145  21145      @annalisewrobel_ awe thank you so much love 💕    fear  0.062
1146  21146                             Omg he kissed her🙈  #w    fear  0.062

[1147 rows x 4 columns]


In [4]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import emoji
import re

In [5]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df1['tweet'] = df1['tweet'].apply(preprocess_tweet)

In [6]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df1['tweet'] = df1['tweet'].apply(remove_mentions)


In [7]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text) 
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df1['tweet'] = df1['tweet'].apply(clean_text)



In [8]:
df1

,Id,tweet,emotion,score
0,20000,i feel like i am drowning #depression #anxiety...,fear,0.979
1,20001,i get so nervous even thinking about talking t...,fear,0.979
2,20002,i lost my blinders #panic,fear,0.975
3,20003,i feel like i am drowning #depression #falure ...,fear,0.938
4,20004,this is the scariest american horror story out...,fear,0.938
...,...,...,...,...
1142,21142,pull over #tonight and make your car #shake,fear,0.104
1143,21143,awe aint he a sweetheart hes adorable,fear,0.083
1144,21144,just had a steak pie supper,fear,0.083
1145,21145,awe thank you so much love,fear,0.062


In [9]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text

In [10]:
df1['tweet'] = df1['tweet'].apply(convert_emoji)

In [11]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df1['tweet'] = df1['tweet'].apply(tokenize_tweets)

In [12]:
df1

,Id,tweet,emotion,score
0,20000,"[i, feel, like, i, am, drowning, #depression, ...",fear,0.979
1,20001,"[i, get, so, nervous, even, thinking, about, t...",fear,0.979
2,20002,"[i, lost, my, blinders, #panic]",fear,0.975
3,20003,"[i, feel, like, i, am, drowning, #depression, ...",fear,0.938
4,20004,"[this, is, the, scariest, american, horror, st...",fear,0.938
...,...,...,...,...
1142,21142,"[pull, over, #tonight, and, make, your, car, #...",fear,0.104
1143,21143,"[awe, aint, he, a, sweetheart, hes, adorable]",fear,0.083
1144,21144,"[just, had, a, steak, pie, supper]",fear,0.083
1145,21145,"[awe, thank, you, so, much, love]",fear,0.062


In [13]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'this', 'all',
    'am', 'or', 'but', 'if', 'my', 'me', 'we', 'us', 'our', 'we', 'up', 'down', 'out', 'just', 'how', 'why',
    'when', 'where', 'here', 'there', 'about', 'more', 'most', 'some', 'any', 'few', 'many', 'much', 'not',
    'only', 'other', 'same', 'such', 'no', 'nor', 'too', 'very', 'can', 'cannot', 'could', 'should', 'would',
    'might', 'must', 'shall', 'will', 'isn', 'hasn', 'doesn', 'haven', 'didn', 'hadn', 'wasn', 'weren',
    'wouldn', 'shouldn', 'ain', 'aren', 'ma'
]

def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens

df1['tweet'] = df1['tweet'].apply(remove_stopwords)



In [14]:
df1


,Id,tweet,emotion,score
0,20000,"[feel, like, drowning, #depression, #anxiety, ...",fear,0.979
1,20001,"[get, nervous, even, thinking, talking, wanna,...",fear,0.979
2,20002,"[lost, blinders, #panic]",fear,0.975
3,20003,"[feel, like, drowning, #depression, #falure, #...",fear,0.938
4,20004,"[scariest, american, horror, story, them, im, ...",fear,0.938
...,...,...,...,...
1142,21142,"[pull, over, #tonight, make, car, #shake]",fear,0.104
1143,21143,"[awe, aint, sweetheart, hes, adorable]",fear,0.083
1144,21144,"[had, steak, pie, supper]",fear,0.083
1145,21145,"[awe, thank, love]",fear,0.062


In [15]:
import torch
from transformers import BertModel, BertTokenizer



model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


def extract_features(tweet):
    
    tokenized_text = ' '.join(tweet)

    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    
    
    with torch.no_grad():
        outputs = model(input_ids)
    
   
    last_hidden_states = outputs.last_hidden_state
    
    features = last_hidden_states.squeeze(0).numpy()
    
    return features


df1['features'] = df1['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
df1

,Id,tweet,emotion,score,features
0,20000,"[feel, like, drowning, #depression, #anxiety, ...",fear,0.979,"[[0.037809342, 0.254312, -0.06298909, -0.16685..."
1,20001,"[get, nervous, even, thinking, talking, wanna,...",fear,0.979,"[[-0.11003713, 0.12191104, 0.17953236, -0.1630..."
2,20002,"[lost, blinders, #panic]",fear,0.975,"[[-0.168674, 0.12842081, -0.0014389177, -0.082..."
3,20003,"[feel, like, drowning, #depression, #falure, #...",fear,0.938,"[[0.06456666, 0.26413164, -0.106887944, -0.123..."
4,20004,"[scariest, american, horror, story, them, im, ...",fear,0.938,"[[-0.21259005, 0.08201837, 0.38263053, 0.04178..."
...,...,...,...,...,...
1142,21142,"[pull, over, #tonight, make, car, #shake]",fear,0.104,"[[-0.1298164, 0.16981113, 0.039434116, 0.13802..."
1143,21143,"[awe, aint, sweetheart, hes, adorable]",fear,0.083,"[[-0.15620765, 0.42569926, -0.16897485, 0.0595..."
1144,21144,"[had, steak, pie, supper]",fear,0.083,"[[-0.33264166, -0.13432035, -0.14950287, 0.282..."
1145,21145,"[awe, thank, love]",fear,0.062,"[[-0.10937966, 0.33537936, 0.05516939, -0.0624..."


In [17]:
df1.features[0].shape

(15, 768)

In [18]:

df1.head(10)

,Id,tweet,emotion,score,features
0,20000,"[feel, like, drowning, #depression, #anxiety, ...",fear,0.979,"[[0.037809342, 0.254312, -0.06298909, -0.16685..."
1,20001,"[get, nervous, even, thinking, talking, wanna,...",fear,0.979,"[[-0.11003713, 0.12191104, 0.17953236, -0.1630..."
2,20002,"[lost, blinders, #panic]",fear,0.975,"[[-0.168674, 0.12842081, -0.0014389177, -0.082..."
3,20003,"[feel, like, drowning, #depression, #falure, #...",fear,0.938,"[[0.06456666, 0.26413164, -0.106887944, -0.123..."
4,20004,"[scariest, american, horror, story, them, im, ...",fear,0.938,"[[-0.21259005, 0.08201837, 0.38263053, 0.04178..."
5,20005,"[nearly, started, crying, having, full, panic,...",fear,0.938,"[[-0.6936238, -0.16869353, 0.12774356, -0.1491..."
6,20006,"[have, finally, tell, therapist, sexuality, la...",fear,0.938,"[[-0.2576109, 0.28046265, -0.046328988, -0.192..."
7,20007,"[dont, think, ive, ever, moved, fast, panic, l...",fear,0.938,"[[-0.27630442, 0.398449, -0.18360381, -0.18951..."
8,20008,"[bus, car, crash, im, still, shaking, bit, wee...",fear,0.938,"[[-0.30670476, 0.43248904, -0.079600625, -0.24..."
9,20009,"[bus, car, crash, im, still, shaking, bit, wee...",fear,0.920,"[[-0.23693994, 0.47858533, -0.1310488, -0.0193..."


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


features = df1['features'].tolist()


padded_features = pad_sequences(features, padding='post')


padded_df = df1.copy()
padded_df['features'] = padded_features.tolist()

In [20]:
X = np.stack(padded_df['features'])

print('Input feature shape:', X.shape)

Input feature shape: (1147, 41, 768)


In [21]:
y = np.array(padded_df['score'])    
print("Input shape:", X.shape)
print("Output shape:", y.shape)

Input shape: (1147, 41, 768)
Output shape: (1147,)


In [23]:
X = np.reshape(X, (1147, 41 * 768))  
y = np.reshape(y, (1147,))

In [24]:
X.shape

(1147, 31488)

In [25]:
Y=df1['score'].copy()

     

In [26]:
import tensorflow as tf

In [49]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(512, input_shape=(31488,), activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))



early_stopping = EarlyStopping(monitor='mean_squared_error', patience=1, mode='min')
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               16122368  
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 256)               131328    
                                                                 
 dropout_16 (Dropout)        (None, 256)               0         
                                                                 
 dense_20 (Dense)            (None, 128)               32896     
                                                                 
 dropout_17 (Dropout)        (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 64)               

In [50]:

df1['tweet'] = df1['tweet'].apply(lambda tokens: ' '.join(tokens))

array = df1['tweet'].values

tensor = tf.convert_to_tensor(array)




In [51]:

# Train the model with the EarlyStopping callback
history = model.fit(X, Y, batch_size=25, epochs=15, shuffle=True, verbose=1, callbacks=[early_stopping])


Epoch 1/15
46/46 [==============================] - 10s 188ms/step - loss: 0.0893 - mean_squared_error: 0.0893
Epoch 2/15
46/46 [==============================] - 9s 185ms/step - loss: 0.0530 - mean_squared_error: 0.0530
Epoch 3/15
46/46 [==============================] - 9s 186ms/step - loss: 0.0385 - mean_squared_error: 0.0385
Epoch 4/15
46/46 [==============================] - 9s 190ms/step - loss: 0.0354 - mean_squared_error: 0.0354
Epoch 5/15
46/46 [==============================] - 8s 181ms/step - loss: 0.0222 - mean_squared_error: 0.0222
Epoch 6/15
46/46 [==============================] - 8s 180ms/step - loss: 0.0190 - mean_squared_error: 0.0190
Epoch 7/15
46/46 [==============================] - 8s 183ms/step - loss: 0.0199 - mean_squared_error: 0.0199


In [52]:
model.save('emointjoy.h5','/Home')

In [53]:
from keras.models import load_model

loaded_model = load_model('emointjoy.h5')

In [54]:
test= pd.read_csv("Desktop/emoint tweet/fear-ratings-0to1.test.target.txt", delimiter='\t', header=None)
test.columns = ['Id', 'tweet', 'emotion', 'score']

In [55]:
test

,Id,tweet,emotion,score
0,21257,#Matthew 25; 1-13\nCould somebody shoot a #vid...,fear,NONE
1,21258,@bkero @whispersystems Which really sucks beca...,fear,NONE
2,21259,Be #afraid of the #quiet ones they are the one...,fear,NONE
3,21260,@riinkanei he's a horrible person and now i ga...,fear,NONE
4,21261,What we fear doing most is usually what we mos...,fear,NONE
...,...,...,...,...
990,22247,"@01Starblazer 9 -9 vs Atlanta this yr, 2 - 11 ...",fear,NONE
991,22248,I'm shaking now.,fear,NONE
992,22249,me: are you guys dating yet #trans #nervous #b...,fear,NONE
993,22250,She: Why are you listening to The Eurythmics?\...,fear,NONE


In [56]:
test= test.drop("score",axis=1)

In [57]:
test

,Id,tweet,emotion
0,21257,#Matthew 25; 1-13\nCould somebody shoot a #vid...,fear
1,21258,@bkero @whispersystems Which really sucks beca...,fear
2,21259,Be #afraid of the #quiet ones they are the one...,fear
3,21260,@riinkanei he's a horrible person and now i ga...,fear
4,21261,What we fear doing most is usually what we mos...,fear
...,...,...,...
990,22247,"@01Starblazer 9 -9 vs Atlanta this yr, 2 - 11 ...",fear
991,22248,I'm shaking now.,fear
992,22249,me: are you guys dating yet #trans #nervous #b...,fear
993,22250,She: Why are you listening to The Eurythmics?\...,fear


In [58]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
test['tweet'] = test['tweet'].apply(preprocess_tweet)

In [59]:
test

,Id,tweet,emotion
0,21257,#matthew 25; 1-13\ncould somebody shoot a #vid...,fear
1,21258,@bkero @whispersystems which really sucks beca...,fear
2,21259,be #afraid of the #quiet ones they are the one...,fear
3,21260,@riinkanei he's a horrible person and now i ga...,fear
4,21261,what we fear doing most is usually what we mos...,fear
...,...,...,...
990,22247,"@01starblazer 9 -9 vs atlanta this yr, 2 - 11 ...",fear
991,22248,i'm shaking now.,fear
992,22249,me: are you guys dating yet #trans #nervous #b...,fear
993,22250,she: why are you listening to the eurythmics?\...,fear


In [60]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

test['tweet'] = test['tweet'].apply(remove_mentions)
test

,Id,tweet,emotion
0,21257,#matthew 25; 1-13\ncould somebody shoot a #vid...,fear
1,21258,which really sucks because typing on a mobil...,fear
2,21259,be #afraid of the #quiet ones they are the one...,fear
3,21260,he's a horrible person and now i gag when i s...,fear
4,21261,what we fear doing most is usually what we mos...,fear
...,...,...,...
990,22247,"9 -9 vs atlanta this yr, 2 - 11 vs rockies an...",fear
991,22248,i'm shaking now.,fear
992,22249,me: are you guys dating yet #trans #nervous #b...,fear
993,22250,she: why are you listening to the eurythmics?\...,fear


In [61]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    return text

test['tweet'] = test['tweet'].apply(clean_text)


In [62]:
test

,Id,tweet,emotion
0,21257,#matthew ncould somebody shoot a #video about ...,fear
1,21258,which really sucks because typing on a mobile ...,fear
2,21259,be #afraid of the #quiet ones they are the one...,fear
3,21260,hes a horrible person and now i gag when i see...,fear
4,21261,what we fear doing most is usually what we mos...,fear
...,...,...,...
990,22247,vs atlanta this yr vs rockies and dbacks this ...,fear
991,22248,im shaking now,fear
992,22249,me are you guys dating yet #trans #nervous #bl...,fear
993,22250,she why are you listening to the eurythmicsnme...,fear


In [63]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text

test['tweet'] = test['tweet'].apply(convert_emoji)

In [64]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

test['tweet'] = test['tweet'].apply(tokenize_tweets)

In [65]:
test

,Id,tweet,emotion
0,21257,"[#matthew, ncould, somebody, shoot, a, #video,...",fear
1,21258,"[which, really, sucks, because, typing, on, a,...",fear
2,21259,"[be, #afraid, of, the, #quiet, ones, they, are...",fear
3,21260,"[hes, a, horrible, person, and, now, i, gag, w...",fear
4,21261,"[what, we, fear, doing, most, is, usually, wha...",fear
...,...,...,...
990,22247,"[vs, atlanta, this, yr, vs, rockies, and, dbac...",fear
991,22248,"[im, shaking, now]",fear
992,22249,"[me, are, you, guys, dating, yet, #trans, #ner...",fear
993,22250,"[she, why, are, you, listening, to, the, euryt...",fear


In [66]:

stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'this', 'all',
    'am', 'or', 'but', 'if', 'my', 'me', 'we', 'us', 'our', 'we', 'up', 'down', 'out', 'just', 'how', 'why',
    'when', 'where', 'here', 'there', 'about', 'more', 'most', 'some', 'any', 'few', 'many', 'much', 'not',
    'only', 'other', 'same', 'such', 'no', 'nor', 'too', 'very', 'can', 'cannot', 'could', 'should', 'would',
    'might', 'must', 'shall', 'will', 'isn', 'hasn', 'doesn', 'haven', 'didn', 'hadn', 'wasn', 'weren',
    'wouldn', 'shouldn', 'ain', 'aren', 'ma'
]

def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens

test['tweet'] = test['tweet'].apply(remove_stopwords)



In [67]:
test

,Id,tweet,emotion
0,21257,"[#matthew, ncould, somebody, shoot, #video, it...",fear
1,21258,"[which, really, sucks, because, typing, mobile...",fear
2,21259,"[#afraid, #quiet, ones, they, ones, who, actua...",fear
3,21260,"[hes, horrible, person, now, gag, see, people,...",fear
4,21261,"[what, fear, doing, usually, what, need, do, t...",fear
...,...,...,...
990,22247,"[vs, atlanta, yr, vs, rockies, dbacks, yr, tha...",fear
991,22248,"[im, shaking, now]",fear
992,22249,"[guys, dating, yet, #trans, #nervous, #blowjob...",fear
993,22250,"[she, listening, eurythmicsnme, polish, gothic...",fear


In [68]:
import torch
from transformers import BertModel, BertTokenizer


model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def extract_features(tweet):
    
    tokenized_text = ' '.join(tweet)
    
    
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    
    with torch.no_grad():
        outputs = model(input_ids)
    
 
    last_hidden_states = outputs.last_hidden_state
    
    features = last_hidden_states.squeeze(0).numpy()
    
    return features


test['features'] = test['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [69]:
test["features"][0].shape

(22, 768)

In [70]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features = test['features'].tolist()

padded_features = pad_sequences(features, padding='post')


padded_df = test.copy()
padded_df['features'] = padded_features.tolist()

In [71]:
X_test= np.stack(padded_df['features'])


print('Input feature shape:', X_test.shape)

Input feature shape: (995, 38, 768)


In [74]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 41

padded_test_data = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')


In [76]:
X = np.reshape(padded_test_data, (995, 41 * 768))  

In [77]:
test_features = np.array(X)

In [78]:
print(test_features.shape)

(995, 31488)


In [79]:
predictions = loaded_model.predict(test_features)

32/32 [==============================] - 0s 11ms/step


In [80]:
print(predictions)

[[0.35848075]
 [0.49418336]
 [0.3316049 ]
 [0.32864428]
 [0.33632284]
 [0.34549308]
 [0.39902824]
 [0.55583555]
 [0.50141495]
 [0.4322241 ]
 [0.5267519 ]
 [0.4741906 ]
 [0.44388682]
 [0.4757291 ]
 [0.4192236 ]
 [0.43171203]
 [0.31003094]
 [0.3322686 ]
 [0.30470473]
 [0.31897712]
 [0.4269125 ]
 [0.2863402 ]
 [0.3659166 ]
 [0.48805356]
 [0.41435736]
 [0.42307842]
 [0.383735  ]
 [0.48033422]
 [0.3701272 ]
 [0.38699406]
 [0.44868565]
 [0.3222087 ]
 [0.3779868 ]
 [0.31596035]
 [0.29360682]
 [0.3806095 ]
 [0.40222389]
 [0.80840164]
 [0.5171999 ]
 [0.43531466]
 [0.41806018]
 [0.40936303]
 [0.39708602]
 [0.35488188]
 [0.34643912]
 [0.42236507]
 [0.3340001 ]
 [0.39021146]
 [0.4741637 ]
 [0.48436433]
 [0.33365536]
 [0.3963105 ]
 [0.33946007]
 [0.3270629 ]
 [0.38040042]
 [0.4648652 ]
 [0.39732796]
 [0.34173644]
 [0.48671055]
 [0.36853778]
 [0.32782137]
 [0.38926625]
 [0.41782165]
 [0.47944754]
 [0.46844542]
 [0.42905098]
 [0.34290272]
 [0.304182  ]
 [0.36181313]
 [0.53571546]
 [0.47912234]
 [0.38

In [81]:
test['score']=pd.DataFrame(predictions)

In [82]:
test=test.drop('features',axis=1)


In [84]:
test

,Id,tweet,emotion,score
0,21257,"[#matthew, ncould, somebody, shoot, #video, it...",fear,0.358481
1,21258,"[which, really, sucks, because, typing, mobile...",fear,0.494183
2,21259,"[#afraid, #quiet, ones, they, ones, who, actua...",fear,0.331605
3,21260,"[hes, horrible, person, now, gag, see, people,...",fear,0.328644
4,21261,"[what, fear, doing, usually, what, need, do, t...",fear,0.336323
...,...,...,...,...
990,22247,"[vs, atlanta, yr, vs, rockies, dbacks, yr, tha...",fear,0.256943
991,22248,"[im, shaking, now]",fear,0.462747
992,22249,"[guys, dating, yet, #trans, #nervous, #blowjob...",fear,0.510608
993,22250,"[she, listening, eurythmicsnme, polish, gothic...",fear,0.348943
